In [115]:
# importing the libraries we need for our Task

import pandas as pd 
import numpy as np
from sqlalchemy import create_engine

In [ ]:
# loading the data 
df = pd.read_csv('../day_4_datasets/sat-results.csv')
df.head(5)

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


In [ ]:
# checking the data 


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB


In [ ]:
# checking columns names 
df.columns

Index(['DBN', 'SCHOOL NAME', 'Num of SAT Test Takers',
       'SAT Critical Reading Avg. Score', 'SAT Math Avg. Score',
       'SAT Writing Avg. Score', 'SAT Critical Readng Avg. Score',
       'internal_school_id', 'contact_extension', 'pct_students_tested',
       'academic_tier_rating'],
      dtype='object')

In [ ]:
# cleaning columns names 

df.columns = df.columns = df.columns.str.strip().str.lower().str.replace('.', '').str.replace(' ', '_')
df.columns

Index(['dbn', 'school_name', 'num_of_sat_test_takers',
       'sat_critical_reading_avg_score', 'sat_math_avg_score',
       'sat_writing_avg_score', 'sat_critical_readng_avg_score',
       'internal_school_id', 'contact_extension', 'pct_students_tested',
       'academic_tier_rating'],
      dtype='object')

In [ ]:
# checking datatypes to decide which columns data types need to be converted to righ data type

df.dtypes

dbn                                object
school_name                        object
num_of_sat_test_takers             object
sat_critical_reading_avg_score     object
sat_math_avg_score                 object
sat_writing_avg_score              object
sat_critical_readng_avg_score      object
internal_school_id                  int64
contact_extension                  object
pct_students_tested                object
academic_tier_rating              float64
dtype: object

In [ ]:
# Data types to Numeric 

col = ['num_of_sat_test_takers', 'sat_critical_reading_avg_score', 
       'sat_math_avg_score', 'sat_writing_avg_score', 
       'sat_critical_readng_avg_score']


df[col] = df[col].apply(pd.to_numeric, errors='coerce')
df.dtypes

dbn                                object
school_name                        object
num_of_sat_test_takers            float64
sat_critical_reading_avg_score    float64
sat_math_avg_score                float64
sat_writing_avg_score             float64
sat_critical_readng_avg_score     float64
internal_school_id                  int64
contact_extension                  object
pct_students_tested                object
academic_tier_rating              float64
dtype: object

In [122]:
# After converting the Data to right DataTypes we still have to work on column  'pct_students_tested', 
# removing the '%' and dividing the column by 100 so we have a percentag we can do calculation on it 

df['pct_students_tested'] = df['pct_students_tested'].str.replace('%', '')
df['pct_students_tested'] = pd.to_numeric(df['pct_students_tested'], errors='coerce')
df['pct_students_tested'] = df['pct_students_tested'] / 100
df['pct_students_tested']

0      0.78
1       NaN
2       NaN
3      0.92
4      0.92
       ... 
488    0.92
489     NaN
490    0.78
491    0.92
492    0.92
Name: pct_students_tested, Length: 493, dtype: float64

In [123]:
# dropping dublicates 

df = df.drop_duplicates()

In [ ]:
# Summing the null values for each column
df.isnull().sum()

dbn                                 0
school_name                         0
num_of_sat_test_takers             57
sat_critical_reading_avg_score     57
sat_math_avg_score                 57
sat_writing_avg_score              57
sat_critical_readng_avg_score      57
internal_school_id                  0
contact_extension                 100
pct_students_tested               115
academic_tier_rating               86
dtype: int64

In [125]:
# cheking how much % each column has Null Values , to decide whether we fill the null values , leave it or drop it. 

df.isnull().mean().sort_values(ascending=False)

pct_students_tested               0.240586
contact_extension                 0.209205
academic_tier_rating              0.179916
num_of_sat_test_takers            0.119247
sat_critical_reading_avg_score    0.119247
sat_math_avg_score                0.119247
sat_writing_avg_score             0.119247
sat_critical_readng_avg_score     0.119247
dbn                               0.000000
school_name                       0.000000
internal_school_id                0.000000
dtype: float64

In [127]:
# filling null values for the score column with median 
score_col = ['academic_tier_rating','num_of_sat_test_takers', 'sat_critical_reading_avg_score', 
             'sat_math_avg_score', 'sat_writing_avg_score',
             'sat_critical_readng_avg_score']
df.loc[:, score_col] = df[score_col].fillna(df[score_col].median())

In [ ]:
# Downlaoding the Data as csv file

simoun_asmar_sat_resaults = df
simoun_asmar_sat_resaults.to_csv('simoun_asmar_sat_results.csv', index=False)

In [130]:
# connecting to the Database

db_url = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

# Create engine
engine = create_engine(db_url)

# Sending DataFrame to the database (inside the `nyc_schools` schema)
simoun_asmar_sat_resaults.to_sql(
    name='simoun_asmar_sat_results',   
    con=engine,
    schema='nyc_schools',              
    if_exists='replace',               
    index=False
)

478